https://github.com/2noise/ChatTTS/blob/main/README_CN.md

https://github.com/2noise/ChatTTS/issues/216

要用chattts环境来跑。

# 第一种跑法

In [ ]:
## 如果pip装chattts库的话，可以用modelscope先把模型下载下来，然后再跑下面代码：
import ChatTTS
import scipy
model_path = '/Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS' ## 下载下来的模型的路径。
chat = ChatTTS.Chat()
chat.load_models(
    vocos_config_path=f"{model_path}/config/vocos.yaml",
    vocos_ckpt_path=f"{model_path}/asset/Vocos.pt",
    dvae_config_path=f"{model_path}/config/dvae.yaml",
    dvae_ckpt_path=f"{model_path}/asset/DVAE.pt",
    gpt_config_path=f"{model_path}/config/gpt.yaml",
    gpt_ckpt_path=f"{model_path}/asset/GPT.pt",
    decoder_config_path=f"{model_path}/config/decoder.yaml", 
    decoder_ckpt_path=f"{model_path}/asset/Decoder.pt", 
    tokenizer_path=f"{model_path}/asset/tokenizer.pt", 
    device='cpu'
)

In [5]:
texts = ["卧槽这个玩意儿有点东西啊[uv_break]，不过暂时也开发不出更多玩法了，仅此而已罢了。能给我来点说唱吗[laugh]", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./tts3.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
 23%|█████████▎                              | 479/2048 [00:17<00:58, 26.82it/s]


In [6]:
texts = ["呦呦切克闹，爱你的猫抛瓦", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./inmp.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
  9%|███▊                                    | 192/2048 [00:06<01:02, 29.55it/s]


In [7]:
texts = ["[laugh]", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./laugh.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
  9%|███▋                                    | 191/2048 [00:06<01:05, 28.43it/s]


# 第二种跑法

In [5]:
## 把ChatTTS库下载到本地解压，然后在ChatTTS文件夹外设置一个代码，删掉pip装的ChatTTS库，
## 跑下一个代码，可能也行。

# Import necessary libraries and configure settings
import torch
import torchaudio
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

# Initialize and load the model: 
chat = ChatTTS.Chat()
chat.load_models(
    # source='local', 
    # local_path="/Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS"
) # Set to True for better performance

# Define the text input for inference (Support Batching)
texts = [
"So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",
]

# Perform inference and play the generated audio
wavs = chat.infer(texts)
Audio(wavs[0], rate=24_000, autoplay=True)

# Save the generated audio 
torchaudio.save("output.wav", torch.from_numpy(wavs[0]), 24000)


INFO:ChatTTS.core:Load from local: /Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS/
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.
INFO:ChatTTS.core:All initialized.


UnboundLocalError: local variable 'Normalizer' referenced before assignment